<a href="https://colab.research.google.com/github/AlecTraas/computational-geo-lab/blob/main/Colab/Kai/quickhull_algo_3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random as rnd

In [ ]:
def main(num_points):
  points = sorted([[rnd.random()*10, rnd.random()*10, rnd.random()*10] for _ in range(num_points)], key=lambda x: x[0])
  np_points = np.array(points)
  fig = plt.figure()
  ax = fig.add_subplot(projection='3d')
  ax.scatter(np_points[:,0], np_points[:,1], np_points[:,2])
  hull = ConvexHull(points)
  hull.quick_hull()

In [ ]:
# I miss structs :(
class ConvexHull():

In [ ]:
  def __init__(self, points):
    self.points = points
    self.hull_points = None
    self.hull_facets = None
    self.hull_ridges = None
    self.neighbors = None

In [ ]:
  def quick_hull(self):
    # I assume, for the sake of simplicity, that no one will input fewer that 4 points
    define_tetrahedron()
    define_neighbors()
    clear_points_within_hull()

In [ ]:
  def define_tetrahedron(self):
    """
    - Get the bounding points (x-min, x-max, y-min, z-max)
    - Connect by distance, filling so that if there is already an ridge the point will look further for a partner.
      We know that each point should have 3 partners, so do not create unnecesary ridges
    - Simply create a facet per trio of distinct points
    """

In [ ]:
  def define_neighbors(self):
    """
    - Iteratively, choose a ridge.
    - Add all facets that contain that ridge to the neighbors list
    """

In [ ]:
  def above_facet(self,face,point):
    # From what I can tell, this is functional
    face = self.hull_facets[face]
    n = np.cross((face[1]-face[0]),(face[2]-face[0]))
    n = n/np.linalg.norm(n)
    d = np.dot(n,(face[0]-point))
    return d > 0

In [ ]:
main(50)